In [1]:
import os
os.chdir("../")
%pwd

'd:\\Machine_Learning\\Consignment-Pricing-Prediction'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
  root_dir: Path
  preprocessor_path: Path
  data_path: Path
  train_path: Path
  test_path: Path

In [3]:
from ConsignmentPricingPrediction.constants import *
from ConsignmentPricingPrediction.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path: CONFIG_FILE_PATH,
            params_file_path: PARAMS_FILE_PATH
            ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)->DataTransformationConfig:
        config = self.config.data_transformation 
        params = self.params

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            preprocessor_path=config.preprocessor_path,
            data_path=config.data_path,
            train_path=config.train_path,
            test_path=config.test_path
        )

        return data_transformation_config

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

class DataTransformation:
    def __init__(
            self,
            config: DataTransformationConfig
            ):
        self.config = config

    def save_split_dataset(self):
        df = pd.read_csv(self.config.data_path)
        train, test = train_test_split(df, random_state=42)
        train.to_csv(self.config.train_path, index=False)
        test.to_csv(self.config.test_path, index=False)
    
    def 